In [ ]:
from datetime import datetime, timedelta
import xarray as xr
from xhistogram.xarray import histogram
import pandas as pd


# We want to make an aggregate of the following set of forecst files, which overlap with 24 hours
u = 'https://thredds.met.no/thredds/dodsC/ww3_4km_archive_files/%Y/%m/%d/ww3_4km_%Y%m%dT00Z.nc'

def open_mfdataset_overlap(url_base, time_series, timedim='time'):
    urls = [t.strftime(u) for t in time_series]
    time_step = time_series[1] - time_series[0]
    print('Opening individual URLs...')
    datasets = [xr.open_dataset(u, chunks='auto').sel({timedim: slice(t, t+time_step-timedelta(seconds=1))})
                for u,t in zip(urls, time_series)]
    print('Concatenating...')    
    ds = xr.concat(datasets, dim=timedim,
                   compat='override', combine_attrs='override', join='override', coords='all')
    return ds

# Making an aggregate for 15 days:
ds = open_mfdataset_overlap(u, pd.date_range('2023-01-01', '2023-01-15'))
print(ds)

# Plotting time series of wind speed at a position
ds.isel({'rlon': 100, 'rlat': 100}).ff.plot()

Opening individual URLs...
